# 3155 Summer 2022 Recs

In [1]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

## Load the data

Past data to be used for giving recs

In [2]:
df_past = pd.read_csv('/mnt/d/sources/reflection_rec_sys/data/data1.csv', encoding = "cp1252")

In [75]:
results = pd.read_csv('../data/Summer 2022/3155/results.csv')

## Sentence Transformers + Pre Processing

In [4]:
len(df_past)

1234

In [5]:
df_past.dropna(inplace=True)
len(df_past)

1188

In [6]:
df_past

,name,semester,week,challenge,solution
0,past student,Fall 2020,1,The biggest challenge was not learning how to ...,I think that instead of dedicating a period of...
3,past student,Fall 2020,1,Getting used to the class structure.,Plan out the day and be aware of all assignmen...
4,past student,Fall 2020,1,"Time management, getting everything organize t...",The way to address the challenge mentioned abo...
7,past student,Fall 2020,1,Time management has been vital especially bein...,"To overcome the threat of being overwhelmed, o..."
8,past student,Fall 2020,1,The biggest challenge this past week was manag...,"Man this is tough, is there anything one man c..."
...,...,...,...,...,...
1229,Austin Holmes,Summer 2022,2,there was a ton of lecturesÂ,monster energy drink
1230,Will Nguyen,Summer 2022,2,Assignment 1 and 2 required a lot of reading.Â,Yes. I did. I tried to read fast because I sti...
1231,Colin Coffie,Summer 2022,2,I think my biggest challenge was finding time ...,This last week I had to sacrifice some sleep t...
1232,Anthony Spears,Summer 2022,2,I think the assignment was the biggest challen...,"Yes, by just doing the work."


### Get embeddings

In [7]:
df_past.reset_index(inplace=True)

In [8]:
CHALLENGE_PROMPT = "challenge"
SOLUTION_PROMPT = "solution"

In [9]:
model = SentenceTransformer('stsb-roberta-large')

In [10]:
sentences_past = df_past[CHALLENGE_PROMPT].values
embeddings_past = model.encode(sentences_past, convert_to_tensor=True)

In [11]:
def get_top_k(challenge, name, K = 5):
    
    #print(name)
    
    embedding = model.encode(challenge, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings_past, embedding).numpy()
    
    new_df = pd.DataFrame()
    #Output the pairs with their score
    for i in range(len(sentences_past)):
        #print("{} \t\t {} \t\t Score: {:.4f}".format(sentences_past[i], challenge, cosine_scores[i][0]))
        new_df = new_df.append({'cos_score':cosine_scores[i][0], 
                                'challenge':sentences_past[i],
                                'solution' : df_past.iloc[i][SOLUTION_PROMPT]}
                               , ignore_index=True)
        
    # Probably could be way more efficient, but...
    # Remove from new_df any pairs that came from the student itself
    # Also assumes indexes are all the same (so will eventually need to be changed)
    #print(name)
    written_by_self = df_past.index[df_past['name'] == name].tolist()
    #print(written_by_self)
    if len(written_by_self) > 0:
        #print(f"{name} had their own removed")
        new_df.drop(index = written_by_self, inplace = True)
        
    #print(new_df)
    # Remove anything with a 1 cosine score (though probably need to look into this more..?)
    # I feel like it's possible for someone else to have the exact same response and
    # a different solution??
    #new_df = new_df[new_df['cos_score']<0.99999]
    #new_df.to_csv('ugh.csv')
        
    output = new_df.nlargest(K, ['cos_score'])
    return output

In [12]:
temp = get_top_k("time management", "bob").reset_index()

/tmp/ipykernel_1207020/491742814.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'cos_score':cosine_scores[i][0],
/tmp/ipykernel_1207020/491742814.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'cos_score':cosine_scores[i][0],
/tmp/ipykernel_1207020/491742814.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'cos_score':cosine_scores[i][0],
/tmp/ipykernel_1207020/491742814.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'cos_score':cosine_scores[i][0],
/tmp/ipykernel_1207020/491742814.py:12: FutureWarning: The frame.append 

In [13]:
print(temp)

   index  cos_score        challenge  \
0    114        1.0  Time Management   
1   1033        1.0  time management   
2    446        1.0  Time management   
3    657        1.0  Time management   
4    191        1.0  Time management   

                                            solution  
0  I've made liberal use of the altered dashboard...  
1         Focus on this class more than I did before  
2  I tried my best, but next time IÃ¢â‚¬â„¢d be m...  
3  I have to do my work early next time to accoun...  
4        Set ample time over the weekend to catch up  


In [88]:
for index, row in results.iterrows():
    
    challenge = row['student\'s reflection']
    name = row['Full Name']
    
    #print(challenge)
    
    temp = get_top_k(challenge, name).reset_index()
    #print(temp)
    
    for i in range(1,6):
        col_name = f"challenge {i}"
        results[col_name][index] = temp['challenge'][i-1]
        
    for i in range(1,6):
        col_name = f"solution {i}"
        results[col_name][index] = temp['solution'][i-1]
    
    for i in range(1,6):
        col_name = f"cos {i}"
        results[col_name][index] = temp['cos_score'][i-1]
        



<ipython-input-88-f2acf11997d8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results[col_name][index] = temp['challenge'][i-1]
<ipython-input-88-f2acf11997d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results[col_name][index] = temp['solution'][i-1]
<ipython-input-88-f2acf11997d8>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results[col_name][index] = temp['cos_score'][i-1]


In [89]:
results.to_csv('../data/Summer 2022/3155 recommendations remove 1.csv')

In [57]:
results

,Email Address,Full Name,Unnamed: 2,student's reflection,challenge 1,challenge 2,challenge 3,challenge 4,challenge 5,solution 1,solution 2,solution 3,solution 4,solution 5,cos 1,cos 2,cos 3,cos 4,cos 5
0,erobin26@uncc.edu,Parker Robinso,2,Making sure I got everything done before midnight,Motivation to finish all my work on time.,working with the team and getting everything d...,Getting all the various assignments done on time.,Getting everything doneÃ‚,Definitely the time management once again. The...,Using my time more wisely and getting better s...,better communication would be great,I can get started on the assignments the secon...,better time managementÃ‚,Continue staying focused and starting early.,0.619145,0.601549,0.582483,0.562906,0.546078
1,yma10@uncc.edu,Kai M,3,assignment 3 I need to reword some of my conte...,The only challenge I had this week was with pa...,Having to watch over 3 hours of videos to take...,My biggest challenge this week was time manage...,The biggest struggle was Lab 3. The hardest th...,Part 3 was the most challenging thing this week.,The way I addressed the challenge above was to...,IÃ‚ hope the questions in the future will be m...,The things I can do to overcome this challenge...,I can try and email Nicholas or another TA if ...,I plan to re-do the third part to see if I mak...,0.583148,0.566718,0.524563,0.522797,0.510753
2,dbakhsha@uncc.edu,Daniel Bakhshande,4,"As of right now, I am stuck on the beautifulso...",The biggest challenge I have face was trying t...,I've had some troubles with gitbash when it co...,getting all the little activities done and cat...,Finishing the lab on time because I started to...,The first part of the lab I am stuck and canno...,I went to TA section and Kiana was able to hel...,For future reference if I do get errors in git...,i dont know. just try to keep up with everything,I can address this challenge and overcome it b...,Start the work a week in advance to give mysel...,0.574370,0.550951,0.530869,0.520080,0.509742
3,mconde2@uncc.edu,Mariam Cond,2,the biggest challenge for my past courses was ...,The biggest challenge this past week was just ...,Biggest challenge this week was to make sure I...,My biggest challenge this week was attempting ...,There were so many assignments to submit and d...,My biggest challenge this week was simply find...,The best way I can address this challenge is t...,I should spend more time for the class.,I shouldn't struggle as badly as I did this pa...,Taking help from the TAs and finishing my home...,I have identified periods in my schedule where...,0.865760,0.860394,0.844706,0.843256,0.836923
4,jkinzung@uncc.edu,Josai Kinzunga Tun,2,Having time to compete them interactively,Understanding how best to answer the discussio...,Definitely the time management once again. The...,Finding group meeting times that worked for ev...,finding time to take some of the longer quizze...,Catching up,Rereading the question a few times and come up...,Continue staying focused and starting early.,We must work efficiently and try to use class ...,"Yes, I made time to take my quizzes by canceli...",Stay up to date with assignments and lectures,0.598164,0.494224,0.473986,0.457219,0.446613
5,aspears8@uncc.edu,Anthony Spear,4,"Yes, I haven't really reached out, but the for...",Literally none of my python softwares work rig...,I think just getting the Part 2 of the lab to ...,I couldnt figure out the way to sort monthly d...,I had a difficult time understanding Modeling ...,Definitely trying to fully understand dash and...,"I've reached out to a TA for help, hopefully t...",I think this is more so on you as an instructo...,Gunna talk to TAs monday.,Speaking with a ta to receive some clarity for...,Well as I mentioned I buried myself in the doc...,0.530987,0.525411,0.525316,0.522300,0.513732
6,mabukha1@uncc.edu,Malik Abukhala,2,Keeping up with the material.,Keeping up with everything,keeping up with everything,Keeping up,Remembering to do it,making sure to stay on